<a href="https://colab.research.google.com/github/OmarShafie/HBKU_Projects/blob/master/Prepare_Fawaed_bio_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon Feb 15 14:33:20 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!pip install transformers
!git clone https://github.com/aub-mind/arabert
!pip install pyarabic
!pip install farasapy
!pip install python-bidi
!pip install arabic_reshaper

     |████████████████████████████████| 1.8MB 9.3MB/s 
     |████████████████████████████████| 890kB 48.5MB/s 
     |████████████████████████████████| 3.2MB 56.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=f4b44f4043259554ec00334715f805dc390ee53871804026e7a8512dde2e6586
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
Cloning into 'arabert'...
remote: Enumerating objects: 206, done.
remote: Counting objects: 100% (206/206), done.
remote: Compressing objects: 100% (151/151), done.
remote: Total 420 (delta 98), reused 152 (delta 49), pack-reused 214
Receiving objects: 100% (420/420), 3.82 MiB | 6.93 MiB/s, done.
Resolving deltas: 100% (221/221), done.
     |████████████████████████████████| 112kB 8.4MB/s 
  Created wheel for pyarabic: filename=PyArabic-0.6.10-cp36-none-any.whl size=113324 sha256=1e26d992d76146b6813eb28929b2e9a83aae8a24223209f28b

In [ ]:
dataset_directory = 'gdrive/MyDrive/HBKU/Research/datasets/tutorial/'
!ls '$dataset_directory'

 bio_sen_tagged_test_farasa.csv    rawi_bio.csv
 bio_sen_tagged_train_farasa.csv   rawi_bio_grade.csv
 bio_tagged.csv			   rawi_bio_grade_old.csv
 bio_tagged_test.csv		   README.md
 bio_tagged_test_farasa.csv	   scrapy.cfg
 bio_tagged_train.csv		   sen_tagged_test.csv
 bio_tagged_train_farasa.csv	   sen_tagged_test_farasa.csv
'data - data.csv'		   sen_tagged_train.csv
 hadith.csv			   sen_tagged_train_farasa.csv
 narrators.csv			   spidyquotes-viewstate.py
 narrators-utf8.csv		   takhreeg.csv
 quotes-1.html			   takhreeg_data.csv
 quotes-2.html			   takhreeg.numbers
 quotes.json			   tutorial
 rawi_bio_category.csv


In [ ]:
!pip install cudf

  ERROR: Failed building wheel for cudf
  Running setup.py clean for cudf
Failed to build cudf
    Running setup.py install for cudf ... error
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-9acg_t3y/cudf/setup.py'"'"'; __file__='"'"'/tmp/pip-install-9acg_t3y/cudf/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' install --record /tmp/pip-record-7rcuio69/install-record.txt --single-version-externally-managed --compile Check the logs for full command output.


In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

#viz
import matplotlib.pyplot as plt
from bidi.algorithm import get_display
import arabic_reshaper
import matplotlib.gridspec as gridspec 
import seaborn as sns

ModuleNotFoundError: ignored

In [ ]:
path_bio_category = dataset_directory + '/rawi_bio_category.csv'
df_bio_category = pd.read_csv(path_bio_category, header=0)
df_bio_category.tail()

In [ ]:
#filter = df_bio_category["faeeda_category"] == 'الإرسال'
#df_bio_category = df_bio_category[filter]

In [ ]:
df_bio_category = df_bio_category[['faeeda_source', 'faeeda_category', 'faeeda_text']]

In [ ]:
path_bio = dataset_directory + '/rawi_bio.csv'
df_bio = pd.read_csv(path_bio, header=0)

df_bio.tail()

In [ ]:
df_bio = df_bio[['targama_id', 'targama_text']]

In [ ]:
#clean data: remove duplicates, beutiful soup
df_bio_category.drop_duplicates(subset=None, keep="first", inplace=True)
df_bio.drop_duplicates(subset=None, keep="first", inplace=True)

In [ ]:
df_bio_category = df_bio_category.fillna(0)
df_bio_category = df_bio_category.astype({'faeeda_source': 'int32'})
df_bio = df_bio.astype({'targama_id': 'int32'})

In [ ]:
df_bio_category['faeeda_text'] = df_bio_category['faeeda_text'].apply(lambda x: BeautifulSoup(x, "lxml").text)
df_bio['targama_text'] = df_bio['targama_text'].apply(lambda x: BeautifulSoup(x, "lxml").text)

In [ ]:
#df_bio_category['faeeda_text'] = df_bio_category['faeeda_text'].apply(lambda x: farasa.segment(x))
#df_bio['targama_text'] = df_bio['targama_text'].apply(lambda x: farasa.segment(x))

In [ ]:
cats = df_bio_category['faeeda_category'].value_counts()
cat_names = list(cats.index)
class_names=list(df_bio_category.groupby('faeeda_category').nunique().index)

In [ ]:
plt.figure()

count = list(cats)

reshaped_categories = [get_display(arabic_reshaper.reshape(cat)) for cat in cat_names]
ax= sns.barplot(y=reshaped_categories, x=count, alpha=0.8)
plt.title('Type ')
plt.ylabel("# per class", fontsize=12)
plt.xlabel('# of Occurrences', fontsize=12)
#adding the text labels
rects = ax.patches
for rect, label in zip(rects, count):
    ax.text(rect.get_x() + rect.get_width() + 10,  rect.get_y() + rect.get_height(), label)

plt.show()

In [ ]:
import re
def split_sentences(text):
  return text.split("/n") #re.split('\.|،',text)

In [ ]:
clean_tags = [0 for label in range(len(cat_names))]
targama_dict = {}
for b in range(len(df_bio)):
  sentences = [s.strip() for s in split_sentences(df_bio["targama_text"].iat[b]) if len(s.strip()) > 0 and len(s.strip()) < 1200]
  labels = [clean_tags.copy() for s in range(len(sentences))]
  targama_dict[df_bio["targama_id"].iat[b]] = (sentences,labels)

In [ ]:
len(df_bio_category["faeeda_source"])

In [ ]:
targama_dict

In [ ]:
def update_tags(mylist, pattern, tag, old_tags):
  new_tags = old_tags[:]
  cat_idx = cat_names.index(tag)
  for i in range(len(mylist)):
      if len(pattern) > 0 and pattern[0] in mylist[i]:
        for j in range(len(pattern)):
          new_tags[i+j][cat_idx] = 1
        return new_tags

In [ ]:
#sanity check
test_row = 42
test_index = df_bio_category["faeeda_source"].iat[test_row]
test_pattern = split_sentences(df_bio_category["faeeda_text"].iat[test_row])
test_list = targama_dict[test_index][0]
for i in test_list:
  print(i) 
update_tags(test_list,test_pattern,df_bio_category["faeeda_category"].iat[test_row], targama_dict[test_index][1])

In [ ]:
missing = 0
found = 0
duplicates = 0
for t in range(len(df_bio_category)):
  index = df_bio_category["faeeda_source"].iat[t]
  if index in targama_dict:
    if not any([any(labels) for labels in targama_dict[index][1]]):
      found += 1
    else:
      duplicates += 1
    update_tags(targama_dict[index][0],
                [s.strip() for s in split_sentences(df_bio_category["faeeda_text"].iat[t]) if len(s.strip()) > 0 and len(s.strip()) < 1200], 
                df_bio_category["faeeda_category"].iat[t],
                targama_dict[index][1])
  else:
    missing += 1
print("missing", missing)
print("found", found)
print("duplicates",duplicates)

In [ ]:
tagged_sen_dict = [(targama_dict[k][0][i],targama_dict[k][1][i]) for k in targama_dict.keys() for i in range(len(targama_dict[k][0]))]

In [ ]:
biography_data = pd.DataFrame.from_dict(targama_dict, orient='index')
sentences_data = pd.DataFrame(tagged_sen_dict, columns=['text', 'labels'])

In [ ]:
sentences_data[[cat for cat in cat_names]] = pd.DataFrame(sentences_data['labels'].to_list())

In [ ]:
sentences_data = sentences_data.drop(columns=['labels'])
sentences_data.head()

In [ ]:
x=sentences_data.iloc[:,1:].sum()
#marking comments without any tags as "clean"
rowsums=sentences_data.iloc[:,1:].sum(axis=1)
sentences_data['clean']=(rowsums==0)
#count number of clean entries
sentences_data['clean'].sum()
print("Total comments = ",len(sentences_data))
print("Total clean comments = ",sentences_data['clean'].sum())
print("Total tags =",x.sum())

In [ ]:
x=sentences_data.iloc[:,1:].sum()
x=rowsums.value_counts()

#plot
plt.figure(figsize=(8,4))
ax = sns.barplot(x.index, x.values, alpha=0.8)
plt.title("Multiple tags per comment")
plt.ylabel('# of Occurrences', fontsize=12)
plt.xlabel('# of tags ', fontsize=12)

#adding the text labels
rects = ax.patches
labels = x.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')

plt.show()

In [ ]:
sentences_data.rename( columns={'':'text'}, inplace=True )
sentences_data

In [ ]:
!pip install ar_wordcloud

In [ ]:
#wordcloud for clean comments
from ar_wordcloud import ArabicWordCloud
for cat in range(len(cat_names)):
  subset=sentences_data[sentences_data[cat_names[cat]]==1]["text"].apply(lambda x: get_display(arabic_reshaper.reshape(x)))
  text=subset.values
  wc= ArabicWordCloud(background_color="black",max_words=2000).generate(" ".join(text))
  plt.figure(figsize=(20,10))
  plt.axis("off")
  plt.title("Words frequented in"+ reshaped_categories[cat] +" fawaed", fontsize=20)
  plt.imshow(wc)
  plt.show()

In [ ]:
sentences_data = sentences_data[sentences_data['clean'] == False]
sentences_data

In [ ]:
from transformers import AutoTokenizer, AutoModel
from arabert.preprocess import never_split_tokens, ArabertPreprocessor
from farasa.segmenter import FarasaSegmenter
import torch

arabert_tokenizer = AutoTokenizer.from_pretrained(
    "aubmindlab/bert-base-arabertv2",
    do_lower_case=False,
    do_basic_tokenize=True,
    never_split=never_split_tokens)
arabert_model = AutoModel.from_pretrained("aubmindlab/bert-base-arabertv2") #you can replace the path here with the folder containing the the pytorch model

farasa_segmenter = FarasaSegmenter(interactive=True)

In [ ]:
model_name = "bert-base-arabertv2"
arabert_prep = ArabertPreprocessor(model_name=model_name, keep_emojis=False)

In [ ]:
from scipy import spatial

def vectorize(s):
  text_preprocessed = arabert_prep.preprocess(s)
  arabert_input = arabert_tokenizer.encode(text_preprocessed)#,add_special_tokens=True, max_length=512, pad_to_max_length=True)
  tensor_input_ids = torch.tensor(arabert_input).unsqueeze(0)
  output = arabert_model(tensor_input_ids)
  embeddings = output[0][0][1:-1]
  return torch.mean(embeddings, dim=0).detach().numpy()

sentences = [
    "قام الطلاب للمعلم",
    "نهض الطلبة للمدرس",
    "ذهب الرجل إلى السوق",
]

vectors_bert = [vectorize(s) for s in sentences]

dist_1 = spatial.distance.cosine(vectors_bert[0], vectors_bert[1])
dist_2 = spatial.distance.cosine(vectors_bert[0], vectors_bert[2])
print('dist_1: {0}, dist_2: {1}'.format(dist_1, dist_2))

In [ ]:
!pip install swifter
import swifter

In [ ]:
sentences_data['embeddings'] = sentences_data['text'].swifter.apply(lambda x: vectorize(x))

In [ ]:
count = 0
for i in range(len(sentences_data['text'])):
  x = sentences_data['text'].iat[i]
  print(i,"-",len(x))
  count+= 1 if len(x)> 1200 else 0
  print(count)
  vectorize(x)

In [ ]:
sentences_data['text'].iat[1471]

In [ ]:
sentences_data

In [ ]:
from sklearn.model_selection import train_test_split
bio_train, bio_test = train_test_split(biography_data, test_size = 0.2, random_state = 42)
sen_train, sen_test = train_test_split(sentences_data, test_size = 0.2, random_state = 42)

In [ ]:
print(len(bio_train),len(bio_test))
print(len(sen_train),len(sen_test))

In [ ]:
bio_train.to_csv('bio_tagged_train.csv')
!cp bio_tagged_train.csv 'gdrive/MyDrive/HBKU/Research/datasets/tutorial/'

bio_test.to_csv('bio_tagged_test.csv')
!cp bio_tagged_test.csv 'gdrive/MyDrive/HBKU/Research/datasets/tutorial/'

'''
sen_train.to_csv('sen_tagged_train.csv')
!cp sen_tagged_train.csv 'gdrive/MyDrive/HBKU/Research/datasets/tutorial/'

sen_test.to_csv('sen_tagged_test.csv')
!cp sen_tagged_test.csv 'gdrive/MyDrive/HBKU/Research/datasets/tutorial/'
'''

To Do:
* Expand to multi-label
* Visualizaiton and ensure correct types
* Clean Data